In [14]:
import os
import pandas as pd
import re
import config_runtime

import config
usuario= config_runtime.usuario
reporte=config_runtime.reporte
path_carpeta =config.step1_extracciones_dmv 
import import_ipynb
import Funciones  # debe contener la función estandarizar_y_unir


In [15]:
import os
import pandas as pd
import config_runtime
import config

# Config
usuario = config_runtime.usuario.strip()
reporte = config_runtime.reporte.strip()
path_carpeta = config.step1_extracciones_dmv

# Filenames esperados
nombres_objetivo = {
    "columnas": f"{usuario} + {reporte} + columnas.xlsx",
    "metricas": f"{usuario} + {reporte} + metricas.xlsx",
    "tablas": f"{usuario} + {reporte} + tablas.xlsx"
}

# Inicializar
df_columnas = pd.DataFrame()
df_metricas = pd.DataFrame()
df_tablas = pd.DataFrame()

# Cargar archivos
for tipo, nombre_archivo in nombres_objetivo.items():
    ruta = os.path.join(path_carpeta, nombre_archivo)
    print(f"[INFO] Buscando: {ruta}")

    if os.path.isfile(ruta):
        try:
            df = pd.read_excel(ruta)
            print(f"[OK] Cargado archivo de {tipo}: {nombre_archivo}")
            if tipo == "columnas":
                df_columnas = df
            elif tipo == "metricas":
                df_metricas = df
            elif tipo == "tablas":
                df_tablas = df
        except Exception as e:
            print(f"[ERROR] Fallo al leer '{nombre_archivo}': {e}")
    else:
        print(f"[WARNING] Archivo no encontrado: {nombre_archivo}")

# Confirmación
print(f"[RESUMEN] Columnas: {len(df_columnas)} | Métricas: {len(df_metricas)} | Tablas: {len(df_tablas)}")

# Unificar columnas + métricas
df_unificado = Funciones.estandarizar_y_unir(df_columnas, df_metricas)

[INFO] Buscando: C:\Users\Administrator\DMV Automation - 10 Jun\DMV Files\testUser + Empower BI V4 + columnas.xlsx


C:\Users\Administrator\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[OK] Cargado archivo de columnas: testUser + Empower BI V4 + columnas.xlsx
[INFO] Buscando: C:\Users\Administrator\DMV Automation - 10 Jun\DMV Files\testUser + Empower BI V4 + metricas.xlsx
[OK] Cargado archivo de metricas: testUser + Empower BI V4 + metricas.xlsx
[INFO] Buscando: C:\Users\Administrator\DMV Automation - 10 Jun\DMV Files\testUser + Empower BI V4 + tablas.xlsx
[OK] Cargado archivo de tablas: testUser + Empower BI V4 + tablas.xlsx
[RESUMEN] Columnas: 315 | Métricas: 7 | Tablas: 21


C:\Users\Administrator\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Administrator\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [16]:
# ✅ quiero traer el campo TableName, tengo Crear columna auxiliar en ambos DataFrames
df_unificado["join_key"] = df_unificado["Reporte"] + df_unificado["TableID"].astype(str)
df_tablas["join_key"] = df_tablas["Reporte"] + df_tablas["ID"].astype(str)

# ✅ Usar .map() para traer el campo 'Name' desde df_mariano_reporte_tablas
df_unificado["Tabla Nombre"] = df_unificado["join_key"].map(
    df_tablas.set_index("join_key")["Name"]
)


df_unificado

,Reporte,ID,TableID,Name,IsHidden,DataType,Expression,ModifiedTime,StructureModifiedTime,DisplayFolder,Origen,join_key,Tabla Nombre
0,Empower BI V4,11,10,RowNumber-2662979B-1795-4F74-8F37-6A1BA8059B61,True,3,NaN,45634.637941,45634.637941,NaN,Columnas,Empower BI V410,Columnas
1,Empower BI V4,140,10,Tipo,True,2,"if(Columnas[Expression]=BLANK(),""Columna"",""Col...",45634.658564,45634.637941,NaN,Columnas,Empower BI V410,Columnas
2,Empower BI V4,141,10,Reporte,False,1,NaN,45634.637941,45634.637941,NaN,Columnas,Empower BI V410,Columnas
3,Empower BI V4,142,10,ID,False,1,NaN,45634.637941,45634.637941,NaN,Columnas,Empower BI V410,Columnas
4,Empower BI V4,143,10,TableID,False,1,NaN,45634.637941,45634.637941,NaN,Columnas,Empower BI V410,Columnas
...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,Empower BI V4,16306,19,Basic Column,False,6,"CALCULATE(COUNT('Columnas Y Metricas'[Name]),'...",45831.108016,45827.974209,NaN,Métricas,Empower BI V419,Columnas Y Metricas
318,Empower BI V4,16308,19,# Measures,False,6,"CALCULATE(COUNT('Columnas Y Metricas'[Name]),'...",45831.108016,45827.975721,NaN,Métricas,Empower BI V419,Columnas Y Metricas
319,Empower BI V4,16310,19,Calculated Column,False,6,"CALCULATE(COUNT('Columnas Y Metricas'[Name]),'...",45831.108016,45827.976179,NaN,Métricas,Empower BI V419,Columnas Y Metricas
320,Empower BI V4,16315,19,Total Columns,False,6,"calculate(COUNT('Columnas Y Metricas'[Name]),o...",45831.108016,45827.993752,NaN,Métricas,Empower BI V419,Columnas Y Metricas


In [17]:
# Definir función para extraer dependencias
def extraer_columnas(expresion, tabla_actual):
    # Buscar patrones de tipo dimension[columna] o [columna] en las expresiones
    patrones = re.findall(r'(\w*)\[(\w+)\]', expresion)
    columnas_referenciadas = []
    columnas_referenciadas_2=[]

    for dimension, columna in patrones:
        if dimension:  # Si hay un prefijo de dimensión
            columnas_referenciadas.append(f"{dimension}.{columna}")
        else:  # Si no hay prefijo de dimensión, pertenece a la tabla actual
            columnas_referenciadas.append(f"{tabla_actual}.{columna}")

    for dimension, columna in patrones:
        if dimension:  # Si hay un prefijo de dimensión
            columnas_referenciadas_2.append(f"{dimension}[{columna}]")
        else:  # Si no hay prefijo de dimensión, pertenece a la tabla actual
            columnas_referenciadas_2.append(f"{tabla_actual}[{columna}]")

    return columnas_referenciadas,columnas_referenciadas_2

In [18]:
# Construir diccionarios de dependencias y expresiones
dependencias = {}
dependencias_2 = {}  # Para las referencias con el formato de corchetes
expresiones = {}

for index, row in df_unificado.iterrows():
    reporte = row['Reporte']
    tabla = row['Tabla Nombre']
    columna = row['Name']
    tabla_columna = f"{tabla}|{columna}"
    print(  reporte,tabla,columna,tabla_columna)
    if reporte not in dependencias:
        dependencias[reporte] = {}
        dependencias_2[reporte] = {}  # Inicializar para dependencias_2
        expresiones[reporte] = {}

    if pd.notna(row['Expression']):
        columnas_referenciadas, columnas_referenciadas_2 = extraer_columnas(row['Expression'], tabla)

        if tabla_columna not in dependencias[reporte]:
            dependencias[reporte][tabla_columna] = []
            dependencias_2[reporte][tabla_columna] = []  # Para dependencias_2

        dependencias[reporte][tabla_columna].extend(columnas_referenciadas)
        dependencias_2[reporte][tabla_columna].extend(columnas_referenciadas_2)  # Para dependencias_2
        expresiones[reporte][tabla_columna] = row['Expression']
    else:
        expresiones[reporte][tabla_columna] = None  # Marca para columnas no calculadas
        dependencias[reporte][tabla_columna] = []  # No tiene dependencias si es directa
        dependencias_2[reporte][tabla_columna] = []  # No tiene dependencias si es directa

Empower BI V4 Columnas RowNumber-2662979B-1795-4F74-8F37-6A1BA8059B61 Columnas|RowNumber-2662979B-1795-4F74-8F37-6A1BA8059B61
Empower BI V4 Columnas Tipo Columnas|Tipo
Empower BI V4 Columnas Reporte Columnas|Reporte
Empower BI V4 Columnas ID Columnas|ID
Empower BI V4 Columnas TableID Columnas|TableID
Empower BI V4 Columnas ExplicitName Columnas|ExplicitName
Empower BI V4 Columnas IsHidden Columnas|IsHidden
Empower BI V4 Columnas ColumnStorageID Columnas|ColumnStorageID
Empower BI V4 Columnas Type Columnas|Type
Empower BI V4 Columnas SourceColumn Columnas|SourceColumn
Empower BI V4 Columnas Expression Columnas|Expression
Empower BI V4 Columnas ModifiedTime Columnas|ModifiedTime
Empower BI V4 Columnas StructureModifiedTime Columnas|StructureModifiedTime
Empower BI V4 Columnas DisplayFolder Columnas|DisplayFolder
Empower BI V4 Columnas Reporte + TableID Columnas|Reporte + TableID
Empower BI V4 Tablas RowNumber-2662979B-1795-4F74-8F37-6A1BA8059B61 Tablas|RowNumber-2662979B-1795-4F74-8F37-6

In [19]:
# Crear el DataFrame final con las columnas solicitadas
data = []

for reporte in dependencias:
    for tabla_columna in dependencias[reporte]:
        try:
            # Safely split into tabla and columna
            tabla, columna = tabla_columna.split("|", maxsplit=1)  # Use maxsplit to avoid extra splits
        except ValueError:
            # Handle cases where the split doesn't produce exactly two parts
            print(f"Warning: Unexpected format in 'tabla_columna': {tabla_columna}")
            tabla, columna = tabla_columna, None  # Assign defaults

        expresion = expresiones[reporte].get(tabla_columna)
        referencias = dependencias[reporte][tabla_columna]
        referencias_2 = dependencias_2[reporte].get(tabla_columna, [])  # Handle missing keys in dependencias_2

        # Determine if it is final
        es_final = 'Si' if expresion is None else 'No'

        # Add the row to the new DataFrame
        data.append({
            'reporte': reporte,
            'Tabla Nombre': tabla,
            'Columna Nombre': columna,
            'Expresión': expresion,
            'Referencias': referencias,
            'Referencias 2': ",".join(referencias_2),  # Join the list as a single string
            'Es Final': es_final
        })

# Crear el DataFrame
df_resultado = pd.DataFrame(data)

# Mostrar los primeros 20 registros para verificar
df_resultado.head(20)


,reporte,Tabla Nombre,Columna Nombre,Expresión,Referencias,Referencias 2,Es Final
0,Empower BI V4,Columnas,RowNumber-2662979B-1795-4F74-8F37-6A1BA8059B61,None,[],,Si
1,Empower BI V4,Columnas,Tipo,"if(Columnas[Expression]=BLANK(),""Columna"",""Col...",[Columnas.Expression],Columnas[Expression],No
2,Empower BI V4,Columnas,Reporte,None,[],,Si
3,Empower BI V4,Columnas,ID,None,[],,Si
4,Empower BI V4,Columnas,TableID,None,[],,Si
5,Empower BI V4,Columnas,ExplicitName,None,[],,Si
6,Empower BI V4,Columnas,IsHidden,None,[],,Si
7,Empower BI V4,Columnas,ColumnStorageID,None,[],,Si
8,Empower BI V4,Columnas,Type,None,[],,Si
9,Empower BI V4,Columnas,SourceColumn,None,[],,Si


In [20]:
from collections.abc import Iterable

# Función que devuelve la expresión de la columna a partir del reporte, tabla y columna
def ObtenerExpresion(df_resultado, reporte, tabla, columna):
    # Filtrar el DataFrame para obtener la expresión de la columna actual
    fila_actual = df_resultado[
        (df_resultado['reporte'] == reporte) &
        (df_resultado['Tabla Nombre'] == tabla) &
        (df_resultado['Columna Nombre'] == columna)
    ]

    # Si se encuentra la fila, devolver la expresión, si no, devolver un mensaje indicando que no se encontró
    if not fila_actual.empty:
        return fila_actual.iloc[0]['Expresión']
    else:
        return "Expresión no encontrada para la columna especificada."

# Función que devuelve un array de referencias a partir del reporte, tabla y columna
def obtener_referencias(df_resultado, reporte, tabla, columna):
    # Filtrar el DataFrame para obtener las dependencias de la columna actual
    fila_actual = df_resultado[
        (df_resultado['reporte'] == reporte) &
        (df_resultado['Tabla Nombre'] == tabla) &
        (df_resultado['Columna Nombre'] == columna)
    ]

    # Si se encuentra la fila, devolver las referencias, si no, devolver un array vacío
    if not fila_actual.empty:
        return list(set(fila_actual.iloc[0]['Referencias']))
    else:
        return None

# Función que devuelve las referencias en formato "dimension[columna]"
def obtener_referencias_2(df_resultado, reporte, tabla, columna):
    # Filtrar el DataFrame para obtener las dependencias formateadas en "Referencias 2"
    fila_actual = df_resultado[
        (df_resultado['reporte'] == reporte) &
        (df_resultado['Tabla Nombre'] == tabla) &
        (df_resultado['Columna Nombre'] == columna)
    ]

    # Si se encuentra la fila, devolver las referencias 2, si no, devolver un array vacío
    if not fila_actual.empty:
        return list(set(fila_actual.iloc[0]['Referencias 2']))
    else:
        return None

# Función recursiva para analizar las dependencias entre columnas y devolver el análisis y las columnas finales
def Recursividad(reporte, tabla, columna, nivel=1, visitados=None, columnasFinales=None, analisis="", lista_columnas_finales=None):
    sangria="          "
    if nivel >= 10:
        return analisis  # Retorna el análisis si el nivel es muy alto


    # Inicializar los parámetros opcionales si son None
    if visitados is None:
        visitados = set()  # Para controlar los nodos ya visitados y evitar bucles
    if columnasFinales is None:
        columnasFinales = set()  # Lista de columnas finales (que no tienen dependencias)
    if lista_columnas_finales is None:
        lista_columnas_finales = []  # Lista para acumular las columnas finales

    # Definir la clave única para la columna actual (tabla.columna)
    clave_columna = f"{tabla}[{columna}]"

    # Caso base: Si la columna ya fue visitada o si es una columna final, terminar la recursividad
    if clave_columna in visitados or clave_columna in columnasFinales:
        return analisis

    # Marcar la columna como visitada
    visitados.add(clave_columna)

    # Obtener su expresión o lógica
    expresion_actual = ObtenerExpresion(df_resultado, reporte, tabla, columna)

    # Obtener las dependencias (referencias a otras columnas) de la columna actual
    dependencias_actuales = obtener_referencias(df_resultado, reporte, tabla, columna)
    dependencias_actuales_2 = obtener_referencias_2(df_resultado, reporte, tabla, columna)  # Obtener referencias 2

    # Agregar información de la columna actual al análisis
    if dependencias_actuales is not None and isinstance(expresion_actual, Iterable):
        try:
            analisis +=      f"Nivel {nivel}:\n"
            analisis +=   f"{clave_columna}=\n"
            analisis +=   f"{expresion_actual}\n"
            analisis +=    f"--Columnas Referenciadas : {', '.join(dependencias_actuales)}\n\n"
        except Exception as e:
            analisis += f"Error procesando {clave_columna}: {str(e)}\n"
            columnasFinales.add(clave_columna)
            lista_columnas_finales.append(clave_columna)
    else:
        # Columnas sin dependencias (finales)
        columnasFinales.add(clave_columna)
        lista_columnas_finales.append(clave_columna)

    # Recorrer las dependencias de la columna y llamar recursivamente
    for referencia in dependencias_actuales or []:
        # Verificar si la referencia sigue el formato "tabla.columna"
        if '.' in referencia:
            tabla_ref, columna_ref = referencia.split('.')
            analisis = Recursividad(reporte, tabla_ref, columna_ref, nivel + 1, visitados, columnasFinales, analisis, lista_columnas_finales)
        else:
            analisis += f"Error: referencia '{referencia}' no tiene formato 'tabla.columna'.\n"

    return analisis

# Ejemplo de uso:
reporte_ejemplo = 'COC and COI'
tabla_ejemplo = 'HR Headcount'
columna_ejemplo = '# Trámites 2_'

# Inicializar string de análisis
analisis_resultado = ""
columnas_finales_lista = []

# Llamar a la función recursiva
analisis_resultado = Recursividad(reporte_ejemplo, tabla_ejemplo, columna_ejemplo, 1, set(), None, analisis_resultado, columnas_finales_lista)

# Concatenar las columnas finales directas de bajada al final del análisis
if columnas_finales_lista:
    analisis_resultado += "\nColumnas Base (Directas de bajada):\n" + ', '.join(set(columnas_finales_lista))

# Mostrar el resultado del análisis
print("Análisis Jerárquico de dependencias:")
print(analisis_resultado)



Análisis Jerárquico de dependencias:

Columnas Base (Directas de bajada):
HR Headcount[# Trámites 2_]


In [21]:
df_resultado.head()

,reporte,Tabla Nombre,Columna Nombre,Expresión,Referencias,Referencias 2,Es Final
0,Empower BI V4,Columnas,RowNumber-2662979B-1795-4F74-8F37-6A1BA8059B61,None,[],,Si
1,Empower BI V4,Columnas,Tipo,"if(Columnas[Expression]=BLANK(),""Columna"",""Col...",[Columnas.Expression],Columnas[Expression],No
2,Empower BI V4,Columnas,Reporte,None,[],,Si
3,Empower BI V4,Columnas,ID,None,[],,Si
4,Empower BI V4,Columnas,TableID,None,[],,Si


In [22]:
import time  # Importar el módulo para medir el tiempo

# Función que realiza el análisis y devuelve el string del análisis con las columnas finales concatenadas
def generar_analisis_completo(row):
    reporte = row['reporte']
    tabla = row['Tabla Nombre']
    columna = row['Columna Nombre']

    # Inicializar variables
    analisis_resultado = ""
    columnas_finales_lista = []

    # Llamar a la función recursiva
    analisis_resultado = Recursividad(reporte, tabla, columna, 1, set(), None, analisis_resultado, columnas_finales_lista)

    # Concatenar las columnas finales al final del análisis
    if columnas_finales_lista:
        analisis_resultado += "\nColumnas Base (Directas de bajada):\n" + ', '.join(set(columnas_finales_lista))

    return analisis_resultado

# Medir el tiempo de ejecución
start_time = time.time()  # Tiempo de inicio

# Aplicar la función a cada fila del DataFrame y agregar la nueva columna "Análisis Completo"
df_resultado['Análisis Completo'] = df_resultado.apply(
    lambda row: generar_analisis_completo(row), axis=1
)

end_time = time.time()  # Tiempo de finalización

# Calcular el tiempo de ejecución
execution_time = end_time - start_time

# Mostrar el DataFrame con la nueva columna "Análisis Completo"
print(df_resultado[['reporte', 'Tabla Nombre', 'Columna Nombre', 'Análisis Completo', 'Referencias 2']])

# Mostrar el tiempo de ejecución
print(f"Tiempo de ejecución: {execution_time:.2f} segundos")


           reporte         Tabla Nombre  \
0    Empower BI V4             Columnas   
1    Empower BI V4             Columnas   
2    Empower BI V4             Columnas   
3    Empower BI V4             Columnas   
4    Empower BI V4             Columnas   
..             ...                  ...   
250  Empower BI V4  Columnas Y Metricas   
251  Empower BI V4  Columnas Y Metricas   
252  Empower BI V4  Columnas Y Metricas   
253  Empower BI V4  Columnas Y Metricas   
254  Empower BI V4           Partitions   

                                     Columna Nombre  \
0    RowNumber-2662979B-1795-4F74-8F37-6A1BA8059B61   
1                                              Tipo   
2                                           Reporte   
3                                                ID   
4                                           TableID   
..                                              ...   
250                                    Basic Column   
251                                      # 

In [23]:
import os
import pandas as pd


# Carpeta destino y nombre del archivo
carpeta_destino = config.step3_analisis_dependencias
os.makedirs(carpeta_destino, exist_ok=True)

nombre_archivo = f"{usuario} + {reporte} + analisis de dependencias.xlsx"
ruta_final = os.path.join(carpeta_destino, nombre_archivo)

# Crear archivo nuevo con hoja 'Dependency Analysis'
with pd.ExcelWriter(ruta_final, engine='openpyxl', mode='w') as writer:
    df_resultado.to_excel(writer, sheet_name="Dependency Analysis", index=False)

print(f"[CREADO] Archivo: {ruta_final}")


[CREADO] Archivo: C:\Users\Administrator\DMV Automation - 10 Jun\DMV Files\testUser + Empower BI V4 + analisis de dependencias.xlsx


In [24]:
df_resultado

,reporte,Tabla Nombre,Columna Nombre,Expresión,Referencias,Referencias 2,Es Final,Análisis Completo
0,Empower BI V4,Columnas,RowNumber-2662979B-1795-4F74-8F37-6A1BA8059B61,None,[],,Si,\nColumnas Base (Directas de bajada):\nColumna...
1,Empower BI V4,Columnas,Tipo,"if(Columnas[Expression]=BLANK(),""Columna"",""Col...",[Columnas.Expression],Columnas[Expression],No,Nivel 1:\nColumnas[Tipo]=\nif(Columnas[Express...
2,Empower BI V4,Columnas,Reporte,None,[],,Si,\nColumnas Base (Directas de bajada):\nColumna...
3,Empower BI V4,Columnas,ID,None,[],,Si,\nColumnas Base (Directas de bajada):\nColumna...
4,Empower BI V4,Columnas,TableID,None,[],,Si,\nColumnas Base (Directas de bajada):\nColumna...
...,...,...,...,...,...,...,...,...
250,Empower BI V4,Columnas Y Metricas,Basic Column,"CALCULATE(COUNT('Columnas Y Metricas'[Name]),'...","[Columnas Y Metricas.Name, Columnas Y Metricas...","Columnas Y Metricas[Name],Columnas Y Metricas[...",No,Nivel 1:\nColumnas Y Metricas[Basic Column]=\n...
251,Empower BI V4,Columnas Y Metricas,# Measures,"CALCULATE(COUNT('Columnas Y Metricas'[Name]),'...","[Columnas Y Metricas.Name, Columnas Y Metricas...","Columnas Y Metricas[Name],Columnas Y Metricas[...",No,Nivel 1:\nColumnas Y Metricas[# Measures]=\nCA...
252,Empower BI V4,Columnas Y Metricas,Calculated Column,"CALCULATE(COUNT('Columnas Y Metricas'[Name]),'...","[Columnas Y Metricas.Name, Columnas Y Metricas...","Columnas Y Metricas[Name],Columnas Y Metricas[...",No,Nivel 1:\nColumnas Y Metricas[Calculated Colum...
253,Empower BI V4,Columnas Y Metricas,Total Columns,"calculate(COUNT('Columnas Y Metricas'[Name]),o...","[Columnas Y Metricas.Name, Columnas Y Metricas...","Columnas Y Metricas[Name],Columnas Y Metricas[...",No,Nivel 1:\nColumnas Y Metricas[Total Columns]=\...
